Input Folder: 'filter_peaks'
Output FOlder: 'brightfield_opt'

Optimizes cells' coordinate system using the brightfield image

In [ ]:
from pathlib import Path

import seaborn as sns
from colicoords import save, load
from dask.distributed import Client
sns.set_style('ticks')

In [ ]:
client = Client('127.0.0.1:8786')
client

In [ ]:
d = Path('../data/filter_peaks').resolve()
files =[f for f in d.iterdir()]

In [ ]:
def worker(fpath):
    cells = load(fpath)
    return {fpath.stem: cells}

In [ ]:
dict_list = client.map(worker, files)

In [ ]:
out = client.gather(dict_list)
out

In [ ]:
peak_cells = {k: v for d in out for k, v in d.items()}

In [ ]:
def optimizer(args):
    name, cells = args
    print(cells, name)
    cells.optimize('brightfield')
    save('cells_bf_opt/' + name + '.hdf5', cells)

In [ ]:
tpl = [(k, v) for k, v in peak_cells.items()]

In [ ]:
ftr = client.map(optimizer, tpl)
ftr
